In [1]:
import pandas as pd

song_data = pd.read_csv('data/song_data_processed.csv')
eurovision = pd.read_csv('data/eurovision_processed.csv')
finals = pd.read_csv('data/finals_processed.csv')
ogae_polls = pd.read_csv('data/ogae_polls.csv')
world_polls = pd.read_csv('data/eurovisionworld_polls.csv')
points2016 = pd.read_csv('data/points_2016.csv')
points2017 = pd.read_csv('data/points_2017.csv')
points2018 = pd.read_csv('data/points_2018.csv')
points2019 = pd.read_csv('data/points_2019.csv')
points2021 = pd.read_csv('data/points_2021.csv')
points2022 = pd.read_csv('data/points_2022.csv')

In [27]:
import pandas as pd

song_df = song_data[['language','country','year']]
song_df = song_df.dropna()
new_rows = []
for index, row in song_df.iterrows():
    if ',\xa0' in row['language']:
        languages = row['language'].split(',\xa0')
        for language in languages:
            new_row = {'language': language, 'country': row['country'], 'year': row['year']}
            new_rows.append(new_row)
new_df = pd.DataFrame(new_rows)
song_df = pd.concat([song_df, new_df])
song_df = song_df[~song_df['language'].str.contains(',\xa0')]
#song_df = song_df.reset_index(drop=True)
print(song_df)

             language      country  year
0   Albanian, English      Albania  2022
1             English       Latvia  2022
2          Lithuanian    Lithuania  2022
3             English  Switzerland  2022
4             Slovene     Slovenia  2022
..                ...          ...   ...
54             Arabic       Israel  2009
55            English     Slovenia  2009
56            Slovene     Slovenia  2009
57            Russian       Russia  2009
58          Ukrainian       Russia  2009

[558 rows x 3 columns]


In [30]:
# Pie chart: language used by country and year (in percentages)
import plotly.express as px
import pandas as pd

song_df = song_data[['language','country','year']]
song_df = song_df.dropna()

years = list(range(2009,2023))
years = [2022]

countries = list(song_data['country'].unique())
countries = [x for x in countries if str(x) != 'nan']
countries = sorted(countries)
countries = ['Finland']

for year in years:
    df = song_df.loc[song_data['year'] == year]
    gender_counts = df['language'].value_counts().reset_index()
    gender_counts.columns = ['language', 'count']
    pie_title = "Gender Distribution of Songs in " + str(year)
    fig = px.pie(gender_counts, values='count', names='language', title=pie_title)
    #fig.show()
    name = "language_pie_year/language_pie_year_" + str(year) + ".html"
    fig.write_html(name) 

for country in countries:
    df = song_df.loc[song_data['country'] == country]
    gender_counts = df['language'].value_counts().reset_index()
    gender_counts.columns = ['language', 'count']
    pie_title = "Gender Distribution of Songs of " + str(country)
    fig = px.pie(gender_counts, values='count', names='language', title=pie_title)
    #fig.show()
    name = "language_pie_country/language_pie_country_" + str(country) + ".html"
    fig.write_html(name) 

In [33]:
# Pie chart: female / male as representative by country and year (in percentages)

import plotly.express as px
import pandas as pd

years = list(range(2009,2023))

countries = list(song_data['country'].unique())
countries = [x for x in countries if str(x) != 'nan']
countries = sorted(countries)

for year in years:
    df = song_data.loc[song_data['year'] == year]
    gender_counts = df['gender'].value_counts().reset_index()
    gender_counts.columns = ['gender', 'count']
    pie_title = "Gender Distribution of Songs in " + str(year)
    fig = px.pie(gender_counts, values='count', names='gender', title=pie_title)
    #fig.show()
    name = "gender_pie_year/gender_pie_year_" + str(year) + ".html"
    fig.write_html(name) 

for country in countries:
    df = song_data.loc[song_data['country'] == country]
    gender_counts = df['gender'].value_counts().reset_index()
    gender_counts.columns = ['gender', 'count']
    pie_title = "Gender Distribution of Songs of " + str(country)
    fig = px.pie(gender_counts, values='count', names='gender', title=pie_title)
    #fig.show()
    name = "gender_pie_country/gender_pie_country_" + str(country) + ".html"
    fig.write_html(name) 

gender_counts = song_data['gender'].value_counts().reset_index()
gender_counts.columns = ['gender', 'count']
pie_title = "Gender Distribution of All Songs"
fig = px.pie(gender_counts, values='count', names='gender', title=pie_title)
#fig.show()
name = "gender_pie_year/gender_pie_year_All.html"
fig.write_html(name) 

name = "gender_pie_country/gender_pie_country_All.html"
fig.write_html(name)